In [5]:

import torch
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7fe82731ca40 to Device at 0x7fe82731d700>

In [19]:
import pandas as pd
import numpy as np
import os

data_dir = "/home/peterr/macocu/task8/transfer/"
train_df = pd.read_csv("37_train_split.csv" )
train_df["split"] = "train"

test_df = pd.read_csv("37_test_split.csv" )
test_df["split"] = "dev"
from ast import literal_eval
train_df["sentence"] = train_df.words.apply(literal_eval).apply(" ".join)
test_df["sentence"] = test_df.words.apply(literal_eval).apply(" ".join)

train_df["path"] = data_dir + train_df["hashname"]
test_df["path"] = data_dir + test_df["hashname"]

In [21]:
tokens = set()
train_df.sentence


0        interpretacija, inzistiranje na tim dezinforma...
1        pomaže. Zato kažem trebamo doći do istine, jer...
2        je nekakav pokazatelj razvijenosti naših lokal...
3        Znači, ovim konačnim prijedlogom zakona, predv...
4        Rezultati i ono što se može vidjeti imamo pozi...
                               ...                        
54243    moći slati naši školarci koji su zainteresiran...
54244    Naime, postojeća zakonska odredba, ograničava ...
54245    invaliditetom osigurali što bolje uvjete život...
54246    tzv. e-planova, to su nove generacije prostorn...
54247    kvalitetan posao, ulaže puno truda i ambicija ...
Name: sentence, Length: 54248, dtype: object

In [2]:
modelname = "38_unnormalised"

# Use old or new vocab?
os.system("cp vocab_300_with_numbers.json vocab.json")

0

In [3]:
train_df["audio"] = train_df.path
test_df["audio"] = test_df.path
import datasets
from datasets import load_dataset, load_metric, Audio

train_dataset = datasets.Dataset.from_pandas(train_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

train_dataset = train_dataset.cast_column("audio", Audio())
test_dataset = test_dataset.cast_column("audio", Audio())

In [4]:


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


# train_mapped = train_dataset.map(
#     prepare_dataset, remove_columns=train_dataset.column_names)
test_mapped = test_dataset.map(
    prepare_dataset, remove_columns=test_dataset.column_names)
print("Data Preparation Complete!")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/500 [00:00<?, ?ex/s]

Data Preparation Complete!


In [12]:

repo_name = modelname
wer_metric = load_metric("wer")
def compute_metrics(pred):
    import numpy as np
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

max_input_length_in_sec = 20
train_mapped = train_mapped.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_extractor()

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=6700//5,
  eval_steps=6700//20,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=5,
  push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_mapped,
    eval_dataset=test_mapped,
    tokenizer=processor.feature_extractor,
)

trainer.train()

  0%|          | 0/55 [00:00<?, ?ba/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it 

Step,Training Loss,Validation Loss


# Looking at output transcriptions:

In [14]:
# %%
# 
from itertools import groupby
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf

##############
# load model & audio and run audio through model
##############
import os
# Use old or new vocab?
# os.system("cp vocab_december.json vocab.json")
# os.system("cp vocab_300.json vocab.json")

# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
#     "/home/peterr/macocu/task8/", unk_token="[UNK]", 
#     pad_token="[PAD]", word_delimiter_token=" ")

# feature_extractor = Wav2Vec2FeatureExtractor(
#     feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

# processor = Wav2Vec2Processor(
#     feature_extractor=feature_extractor, tokenizer=tokenizer)



#model_name = '/home/peterr/macocu/task8/300_/checkpoint-2800'
# model_name = '/home/peterr/macocu/task8/200_/checkpoint-6000'
model_name = '/home/peterr/macocu/task8/19_unnormalised/checkpoint-3028'

model = Wav2Vec2ForCTC.from_pretrained(model_name).cuda()

def get_transcript(audio_filepath:str):
    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()

    ##############
    # this is where the logic starts to get the start and end timestamp for each word
    ##############
    words = [w for w in transcription.split(' ') if len(w) > 0]
    predicted_ids = predicted_ids[0].tolist()
    duration_sec = input_values.shape[1] / sample_rate


    ids_w_time = [(i / len(predicted_ids) * duration_sec, _id) for i, _id in enumerate(predicted_ids)]
    # remove entries which are just "padding" (i.e. no characers are recognized)
    ids_w_time = [i for i in ids_w_time if i[1] != processor.tokenizer.pad_token_id]
    # now split the ids into groups of ids where each group represents a word
    split_ids_w_time = [list(group) for k, group
                        in groupby(ids_w_time, lambda x: x[1] == processor.tokenizer.word_delimiter_token_id)
                        if not k]
    assert len(split_ids_w_time) == len(words)  # make sure that there are the same number of id-groups as words. Otherwise something is wrong

    word_start_times = []
    word_end_times = []
    for cur_ids_w_time, cur_word in zip(split_ids_w_time, words):
        _times = [_time for _time, _id in cur_ids_w_time]
        word_start_times.append(min(_times))
        word_end_times.append(max(_times))

    return words, word_start_times, word_end_times



loading configuration file /home/peterr/macocu/task8/19_unnormalised/checkpoint-3028/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",

In [15]:
" ".join(get_transcript("/home/peterr/macocu/task8/transfer_10/sPNuDebATRM_1853.82-1873.66.wav")[0])

'konačni prijedlog zakona o izmjenama i dopunama zakona o doprinosima drugo čitanje pzbr19 i konačni prijedlog zakona o porezu na dohodak drugo čitanje pzbr23. predlagatelj je vlada rpublike hrvatske na temelju članka 85. ustava rh i članka 172. poslovnika'

In [16]:
dev = pd.read_csv("17_dev.csv")
test= pd.read_csv("17_test.csv")
dev.head()

,hashname,human_transcript,unnormalized_transcript,18_normalised_output,19_unnormalised_output
0,iQoZMemua98_11302.86-11322.81.wav,kome je znači sve dužna i na kraju ono najškak...,kome je znači sve dužna i na kraju ono najškak...,kome znači sve dužna i na kraju ono naješkaklj...,kome znači sve dužna i na kraju ono naješkaklj...
1,UPHLmUzadcg_19923.4-19940.79.wav,jel ako to jednoj maloj uskoj skupini građana ...,jel ako to jednoj maloj uskoj skupini građana ...,jel ako to jednoj maloj uskoj skupini građana ...,jel ako to jednoj maloj uskoj skupini građana ...
2,AAT6OADGdNM_5452.46-5470.19.wav,voda da se ulaže i otklone se ti gubitci kako ...,voda da se ulaže i otklone se ti gubitci kako ...,voda da se ulaže i otklone se ti gubici kako o...,voda da se ulaže i otklone se ti gubici kako o...
3,5T4HXO1WsDU_14739.94-14758.7.wav,što se tiče ove druge vaše opaske ja mislim da...,što se tiče ove druge vaše opaske ja mislim da...,što se tiče ove druge vaše opaske ja mislim da...,što se tiče ove druge vaše opaske ja mislim da...
4,CYBkG0--J8c_11204.74-11224.68.wav,smještene samo starije osobe nego je doista je...,smještene samo starije osobe nego je doista je...,smještene samo starije osobe nego je doista je...,smještene samo starije osobe nego je doista je...


In [10]:
dev[modelname+"_output"] = dev.hashname.apply(
    lambda s: " ".join(get_transcript(f"{data_dir}{s}")[0])
)

In [11]:
test= pd.read_csv("17_test.csv")
test[modelname+"_output"] = test.hashname.apply(
    lambda s: " ".join(get_transcript(f"{data_dir}{s}")[0])
)

In [12]:
test[["human_transcript", "18_normalised_output"]].sample(10)

,human_transcript,18_normalised_output
292,hoću istaknuti jedan podatak koji je ovdje vrl...,je hoću istaknuti jedan podatak koji je ovdje ...
488,zašto pa zato jer su im dostupne one informaci...,zašto a zato jer su im dostupne one informacij...
386,sve što ste rekli utemeljeno je na činjenicama...,sve što ste rekli utemeljeno je na činjenicama...
380,su u splitu razlika između cijene koja je bila...,su u splitu jer je u splitu razlika između cij...
409,poštovani potpredsjedniče poštovane zastupnice...,poštovani potpredsjedniče poštovane zastupnice...
490,dužnike koji imaju kamatnu stopu vezanu u euri...,dužnike koji imaju kamatnu stopu vezano uz eur...
321,i je li to ovo izvješće o kojem vi pričate kak...,jeli to ovo izvješće o kojem vi pričate kako s...
363,definitivno u takvim krajevima možda mogla pob...,definitivno u takvim krajevima možda mogla pob...
472,donese zakone i da ćemo zapravo biti na istoj ...,donese zakoni i da ćemo zapravo biti na istojs...
333,i promijenjen je u dva navrata zakonom o najmu...,i promijenjenje u dva navrata zakonom najmu st...


In [13]:
dev.to_csv("17_dev.csv", index=False)
test.to_csv("17_test.csv", index=False)